<a href="https://colab.research.google.com/github/merhaines/Deep-Learning-Project/blob/main/icepredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Data

The data is found at https://nsidc.org/data/g10010/versions/2#anchor-1. The dataset consists of ice concentration data for each pixel in a 240 x 1140 grid of the Arctic. Ice concentration is between 0-100, and land pixels are set to 120. For the purpose of this analysis, any concentration greater than or equal to 15 is considered a pixel covered with sea ice, and any pixel with a concentration less than 15 is considered open water.

**To download the data**, go to our github repository and download the folder [G10010_SIBT1850_V2](https://github.com/merhaines/Deep-Learning-Project/tree/main/G10010_SIBT1850_V2)

# Imports

In [ ]:
! pip install netcdf4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00


In [ ]:
import netCDF4 as nc
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dense


# Preprocessing

In [ ]:
# mount drive and access dataset
fp = '/content/drive/MyDrive/sea ice predictor/G10010_SIBT1850_V2/G10010_sibt1850_v2.0.nc' # where folder is saved in drive
ds = nc.Dataset(fp)
siconc = ds['seaice_conc'] 

In [ ]:
# grab months
n_months_in_dataset = len(siconc[:])
n_rows = len(siconc[1, :])
n_cols = len(siconc[1, 1, :])
forecast = 3; # 3 month forecast
n_months_input = 3*12;

X_input = siconc[(n_months_in_dataset - n_months_input - forecast):-forecast]
Y_input = siconc[(n_months_in_dataset - n_months_input): ]

# set land pixels to 0
X_input[X_input == 120] = 0
Y_input[Y_input == 120] = 0

# normalize
X_input = X_input/100
Y_input = Y_input/100

# make Y_labels 0 (water) or 1 (ice)
Y_input[Y_input < 0.15] = 0.0
Y_input[Y_input >= 0.15] = 1.0

X_input = X_input.data
Y_input = Y_input.data
Y_input = Y_input.astype(int)

# convert Y_labels to 1D matrix
Y_input = Y_input.reshape(36, 240*1440)


In [ ]:
Y_input.shape
a = [[1, 1, 1],
    [1, 5, 1],
    [3, 1, 8]]
a1 = a * 2
a2 = a * 20
b = [a, a1, a2]
type(b)


list

In [ ]:
# split into training and test data
X_train = X_input[0:30, :, :]
Y_train = Y_input[0:30]

X_test = X_input[30:, :, :]
Y_test = Y_input[30:]

# Define Model

In [ ]:
# create the actual neural net

# Define the model architecture
model = Sequential()

# Add a convolutional layer with 32 filters, 3x3 kernel size, and ReLU activation
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(240,1440,1)))

# Add a max pooling layer with 2x2 pool size
model.add(MaxPooling2D(pool_size=(2,2)))

# Add another convolutional layer with 64 filters, 3x3 kernel size, and ReLU activation
model.add(Conv2D(2, (3,3), activation='relu'))

# Add another max pooling layer with 2x2 pool size
model.add(MaxPooling2D(pool_size=(2,2)))

# Add a flatten layer to convert the 2D output from the convolutional layers into a 1D feature vector
model.add(Flatten())

# Add a fully connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

# Add an output layer with a single unit and sigmoid activation for binary classification
model.add(Dense(units = 240*1440, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 238, 1438, 32)     320       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 119, 719, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 117, 717, 2)       578       
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 58, 358, 2)       0         
 g2D)                                                            
                                                                 
 flatten_14 (Flatten)        (None, 41528)             0         
                                                                 
 dense_19 (Dense)            (None, 128)             

# Train

In [ ]:
# train the model
history = model.fit(X_train, Y_train, epochs=30, validation_data=(X_test, Y_test))

Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 0.6931 - accuracy: 0.0000e+00 - val_loss: 0.6920 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 319ms/step - loss: 0.6919 - accuracy: 0.0000e+00 - val_loss: 0.6757 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 347ms/step - loss: 0.6737 - accuracy: 0.0000e+00 - val_loss: 0.6046 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 324ms/step - loss: 0.5940 - accuracy: 0.0000e+00 - val_loss: 0.4503 - val_accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 331ms/step - loss: 0.4213 - accuracy: 0.0000e+00 - val_loss: 0.2640 - val_accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 316ms/step - loss: 0.2144 - accuracy: 0.0000e+00 - val_loss: 0.1688 - val_accuracy: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 309ms/step - loss: 0.1061 - accuracy: 0.0000e+00 - val_loss: 0